In [1]:
import os,cv2,imutils
import numpy as np
from scipy.spatial import distance

In [2]:
class DETECTION:
    def __init__(self):
        self.Labels=open("coco.names",'r').read().split("\n")
        self.Network=cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")
        self.UCOL=[self.Network.getLayerNames()[i[0]-1] for i in self.Network.getUnconnectedOutLayers()]
    def Detect_People(self,frame,person_id=0):
        H,W=frame.shape[:2]
        Blob=cv2.dnn.blobFromImage(frame,1/255.0,(416,416),swapRB=True,crop=False)
        self.Network.setInput(Blob)
        Output_Layers=self.Network.forward(self.UCOL)
        BOXES,CONFIDENCES,CENTROIDS=[],[],[]
        for outputs in Output_Layers:
            for detected in outputs:
                scores = detected[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if classID==person_id and confidence>=0.3:
                    BOX=detected[0:4]*np.array([W,H,W,H])
                    (X_CENTER,Y_CENTER,Width,Height)=BOX.astype("int")
                    X=(X_CENTER-(Width/2))
                    Y=(Y_CENTER-(Height/2))
                    BOXES.append([int(X),int(Y),int(Width),int(Height)])
                    CENTROIDS.append((X_CENTER,Y_CENTER))
                    CONFIDENCES.append(float(confidence))
        idx=cv2.dnn.NMSBoxes(BOXES, CONFIDENCES, 0.3, 0.3)
        result=[]
        if len(idx)>0:
            for i in idx.flatten():
                x,y=BOXES[i][0],BOXES[i][1]
                w,h=BOXES[i][2],BOXES[i][3]
                r=(CONFIDENCES[i],(x,y,x+w,y+h),CENTROIDS[i])
                result.append(r)
        return result
    def VIDEO_STREAM(self,video):
        V=cv2.VideoCapture(video)
        print("START DETECTING")
        while(True):
            _,frame=V.read()
            frame=imutils.resize(frame,width=700)
            RESULT=self.Detect_People(frame,self.Labels.index("person"))
            voilate=set()
            try:
                if len(RESULT)>=2:
                    centroids=[i[2] for i in RESULT]
                    dists=distance.cdist(centroids,centroids,metric="euclidean")
                    for i in range(0,dists.shape[0]):
                        for j in range(i+1,dists.shape[1]):
                            if dists[i,j]<100:
                                voilate.add(i)
                                voilate.add(j)
                for i,(prob,box,cent) in enumerate(RESULT):
                    x,y,end_x,end_y=box
                    x_cent,y_cent=cent
                    color=(0,255,0)
                    if i in voilate:
                         color=(0,0,255)
                    cv2.rectangle(frame,(x,y),(end_x,end_y),color,2)

                cv2.imshow("video",frame)
                if cv2.waitKey(1) == ord("q"):
                    break
            except:
                pass
    def DESTROY_WINDOW(self):
        cv2.destroyAllWindows() 
            

In [3]:
RUN=DETECTION()

In [ ]:
RUN.VIDEO_STREAM("pedestrians.mp4")
RUN.DESTROY_WINDOW()

START DETECTING


In [ ]:
# RUN.VIDEO_STREAM("https://192.168.43.1:8080/video")